In [ ]:
#distribute to multiple nodes
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor 
from multiprocessing.pool import ThreadPool
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
#extract inputs 
parent_batch = int(dbutils.widgets.get("parent_batch"))


In [ ]:
#parition into 5 partitions 
zipcode = pd.read_csv("/dbfs/FileStore/ZipCodes_Redfin_Scraper.csv")

unique_zipcodes = [str(i) for i in zipcode["ZipCode"].unique()][:1000]

batch_zipcodes = np.array_split(unique_zipcodes, 3)

result = np.array_split(batch_zipcodes[parent_batch], 3)

list_of_batches = [result[0], result[1], result[2]]

final_list = [",".join(i) for i in list_of_batches]
argument_list = []
for i in range(len(final_list)):
    argument_list.append((final_list[i], i, parent_batch))
    

In [ ]:
def processNotebook(list_of_zipcodes, batch_number, parent_batch):
    dbutils.notebook.run(
        path = "/Shared/test/test_tony",
        timeout_seconds = 86400,
        arguments = {
            "list_of_zipcode":list_of_zipcodes,
            "batch_number":batch_number, 
            "parent_number": parent_batch
                    }
    )
    
def helper(numbers):
    processNotebook(numbers[0], numbers[1], numbers[2])

In [ ]:
with ThreadPoolExecutor() as executor:
    for _ in executor.map(helper, argument_list):
        pass

In [ ]:
# %py 

# import pandas as pd

# columns = fieldnames = [
#             "address",
#             "State_zipcode",
#             "last_checked",
#             "last_updated",
#             "price",
#             "bed", 
#             "bath",
#             "square_footage",
#             "description",
#             "status",
#             "time_on_redfin",
#             "property_type",
#             "HOA_dues",
#             "commission",
#             "year_built",
#             "community",
#             "lot_size",
#             "MLS",
#             "list_price",
#             "ppsf",
#             "redfin_est",
#             "style",
#             "lat",
#             "long",
#             "agent_name",
#             "agent_company",
                    
# #                     "school_name": school_name, 
# #                     "school_review": school_review,
# #                     "school_num_students": school_num_students,
#             "url"
#                      ]

# df = pd.read_csv(
#     "/tmp/2021-12-13.csv",
#     names=columns
#                 )